In [ ]:
import os
from pandas import DataFrame 
import networkx as nx

os.chdir("/home/yizhou7/mariposa/")

from debugger.debugger import Debugger3
from debugger.tree_node import NodeRef
from evaluator import BenchViewer
from evaluator import DebuggerStatus
from benchmark_consts import *
from utils.system_utils import list_smt2_files

In [10]:
mv = BenchViewer(UNSTABLE_MARIPOSA)
# vv = BenchViewer(UNSTABLE_VERUS)

[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--MapSpec-TSJMap_Refines_ThreeStateVersionedMap.i.dfy.Impl__TSJMap__Refines__ThreeStateVersionedMap.__default.RefinesStutter.smt2 [INFO] [init] query path: data/projs/bench_unstable/base.z3/d_komodo--verified-svc_handlers.gen.dfyCheckWellformed___module.__default.va__lemma__svc__map__data__success.smt2 [INFO] [init] query path: data/projs/bench_unstable/base.z3/d_komodo--verified-ptebits.i.dfyImpl___module.__default.lemma__l1ptesmatch.smt2 [INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-BucketModel.i.dfy.Impl__BucketModel.__default.flushedMapPopFront.smt2 



[INFO] [init] dbg root: dbg/f8e63dc317/ [INFO] [init] dbg root: dbg/95fa9421b7/ [INFO] [init] dbg root: dbg/6b4aac4136/ 


[INFO] [init] dbg root: dbg/ffb00a9e92/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--lib-Base-total_order_impl.i.dfy.Impl__Total__Order__Impl.__default.ArrayLargestLtePlus1.smt2 
[INFO] 

[ERROR] no such project group singleton_d874a82c3a 


[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_komodo--verified-exception_handlers.gen.dfyImpl___module.__default.va__lemma__abort__handler.smt2 [INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--Impl-JournalistModel.i.dfy.CheckWellformed__JournalistModel.__default.parseJournals.smt2 

[INFO] [init] dbg root: dbg/751af3ffba/ [INFO] [init] dbg root: dbg/0c00bb9db6/ 

[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-PackedStringArray.i.dfy.Impl__PackedStringArray.__default.BinarySearchIndexOfFirstKeyGt.smt2 
[INFO] [init] dbg root: dbg/39c8cf4676/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--Betree-Betree_Refines_Map.i.dfy.Impl__Betree__Refines__Map.__default.RedirectRefinesMap.smt2 
[INFO] [init] dbg root: dbg/3366a80004/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--lib-Buckets-PackedStringArray.i.dfy.Impl__PackedStringArray.DynamicPsa.AppendSeq.smt2 
[INFO] [init] dbg root

[ERROR] no such project group singleton_e93855aadd 


[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--Impl-JournalistImpl.i.dfy.Impl__JournalistImpl.Journalist.append.smt2 
[INFO] [init] dbg root: dbg/f9f9100cfb/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--PivotBetree-PivotBetreeSpecRefinement.i.dfy.Impl__PivotBetreeSpecRefinement.__default.InUpperBoundAndNot.smt2 
[INFO] [init] dbg root: dbg/79fffdb8cb/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--Impl-InsertImpl.i.dfy.Impl__InsertImpl.__default.insert.smt2 
[INFO] [init] dbg root: dbg/caebe30494/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--BlockCacheSystem-BlockSystem.i.dfy.Impl__BlockSystem.__default.DirtyStepPreservesInv.smt2 
[INFO] [init] dbg root: dbg/fae7756eb4/ 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proof available 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-BucketWeights.i.dfy.

...

[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-KMBPKVOps.i.dfy.Impl__KMBPKVOps.__default.FromPkv.smt2 
[INFO] [init] dbg root: dbg/87abcdd247/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_komodo--verified-sha-sha256-one-block.gen.dfyImpl___module.__default.va__abstract__update__Hs.smt2 
[INFO] [init] dbg root: dbg/9796f7a39f/ [INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.CheckWellformed__ByteSystem.__default.DiskInternalStepPreservesInv.smt2 

[INFO] [init] dbg root: dbg/2ead327dad/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--lib-Base-BitsetLemmas.i.dfy.Impl__BitsetLemmas.__default.set__bit__to__0__self__uint64.smt2 
[INFO] [init] dbg root: dbg/0defc27a3b/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-PackedStringArray.i.dfy.Impl__PackedStringArray.__default.BinarySearchIndexOfFirstKeyGte.smt2 
[INFO] [init] dbg r

[ERROR] no such project group singleton_f6db5998ec 


[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--Impl-QueryModel.i.dfy.Impl__QueryModel.__default.AugmentLookup.smt2 
[INFO] [init] dbg root: dbg/d8fb40a98b/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-BucketsLib.i.dfy.Impl__BucketsLib.__default.BInternalWFBucket.smt2 
[INFO] [init] dbg root: dbg/4d7e86f7da/ [INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.Impl__ByteSystem.__default.DiskInternalStepPreservesInv.smt2 

[INFO] [init] dbg root: dbg/519d87022d/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--lib-Marshalling-GenericMarshalling.i.dfy.Impl__GenericMarshalling.__default.MarshallByteArray.smt2 
[INFO] [init] dbg root: dbg/d727e16c9e/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_komodo--verified-map_common.gen.dfyImpl___module.__default.va__lemma__is__valid__mapping__target.smt2 
[INFO] [init] dbg root: dbg/d75368331e/ [INFO]

[ERROR] no such project group singleton_e41d0ee45e 


[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--lib-Base-Multisets.i.dfy.Impl__Multisets.__default.ApplySeq.smt2 [INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--lib-Marshalling-Seqs.i.dfy.Impl__Collections____Seqs__i.__default.lemma__SeqCatRev__adds.smt2 

[INFO] [init] dbg root: dbg/60095b1a59/ [INFO] [init] dbg root: dbg/36a1859f27/ 

[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_lvbkv--lib-Buckets-BucketImpl.i.dfy.Impl__BucketImpl.__default.PartialFlush.smt2 
[INFO] [init] dbg root: dbg/47775b0cf2/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--Impl-JournalistImpl.i.dfy.Impl__JournalistImpl.Journalist.parseJournals.smt2 
[INFO] [init] dbg root: dbg/059d8a76e6/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_komodo--verified-psrbits.i.dfyImpl___module.__default.lemma__update__psr__f.smt2 
[INFO] [init] dbg root: dbg/0b346d0f87/ 
[INFO] [init] query path: data/projs/bench_unstable/base.z3/d

In [11]:
mv.status.print_status()

| category                                      |   count | percentage   |
|-----------------------------------------------|---------|--------------|
| DebuggerStatus.FINISHED                       |     451 | 82.75 %      |
| DebuggerStatus.NO_PROOF                       |      59 | 10.83 %      |
| DebuggerStatus.SINGLETON_CREATION_FAILED      |      19 | 3.49 %       |
| DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED |       9 | 1.65 %       |
| DebuggerStatus.SINGLETON_NOT_RAN              |       6 | 1.1 %        |
| DebuggerStatus.FILTERED_NOT_RAN               |       1 | 0.18 %       |
| total                                         |     545 | 100.00 %     |


In [3]:
for q in mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]:
    print("./src/debugger3.py --create-singleton -i", q)

./src/debugger3.py --create-singleton -i data/projs/bench_unstable/base.z3/d_fvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.Impl__ByteSystem.__default.ReqWriteStepPreservesInv.smt2
./src/debugger3.py --create-singleton -i data/projs/bench_unstable/base.z3/d_komodo--verified-alloc_spare.gen.dfyImpl___module.__default.va__lemma__kom__smc__alloc__spare.smt2
./src/debugger3.py --create-singleton -i data/projs/bench_unstable/base.z3/d_fvbkv--Impl-CoordinationImpl.i.dfy.Impl__CoordinationImpl.__default.initialization.smt2
./src/debugger3.py --create-singleton -i data/projs/bench_unstable/base.z3/d_fvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.Impl__ByteSystem.__default.ReqReadStepPreservesInv.smt2
./src/debugger3.py --create-singleton -i data/projs/bench_unstable/base.z3/d_fvbkv--lib-DataStructures-LruImpl.i.dfy.Impl__LruImpl.LruImplQueue.Remove.smt2
./src/debugger3.py --create-singleton -i data/projs/bench_unstable/base.z3/d_fvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.Impl__ByteSystem.__defau

In [12]:
from utils.system_utils import list_smt2_files

# for q in mv.status[DebuggerStatus.SINGLETON_NOT_RAN]:
#     if len(list_smt2_files(mv[q].singleton_dir)) == 0:
#         continue
#     print(mv[q].get_command_to_run() + " --clear")
#     print("./src/analysis_wizard.py singleton -e verify -s z3_4_13_0 -i ", mv[q].singleton_dir)
#     print("python3 src/carve_and_rerun.py", mv[q].singleton_filtered_dir)

print(len(mv.fixable)/len(mv.status[DebuggerStatus.FINISHED]))

0.7583148558758315


In [ ]:
bad_min_ranks = dict()

for q in mv.fixable:
    r = mv.reviewers[q]
    report = r.build_report()
    indices = report.freq.loc[report.freq["qname"].isin(report.stabilized["qname"])].index
    report.freq["rank"] = report.freq["trace_count"].rank(method='min', ascending=False)
    min_rank = report.freq.loc[indices]["rank"].min()
    max_rank = report.freq.loc[indices]["rank"].max()
    if min_rank <= 10:
        continue
    bad_min_ranks[q] = (min_rank, len(indices))

for q, (rank, feasible)  in bad_min_ranks.items():
    print(rank, feasible, q)

In [ ]:
for q in mv.status[DebuggerStatus.NO_PROOF]:
    print('"' + mv[q].name_hash + '"', end=", ")

In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]:
    print(mv[q].get_command_to_run())

In [ ]:

from evaluator import try_get_singleton_analyzer


maybe_missing = []
for q in mv.unfixable:
    rev = mv.reviewers[q]
    rep = rev.build_report()
    ba = try_get_singleton_analyzer(rev)


[WARN] [eval] be808dd0c3 tested count 997 < registered 998 
[WARN] [eval] ab52dadac1 tested count 69 < registered 1946 
482 1395 data/projs/bench_unstable/base.z3/d_lvbkv--Impl-MainHandlers.i.dfy.Impl__MainHandlers.__default.SystemRefinesCrashSafeMapInit.smt2
